In [1]:
import pandas as pd
from helper_functions import get_master_df, save_df, drop_extraneous_col

In [2]:
df = get_master_df()

In [3]:
drop_extraneous_col(df)

In [4]:
team_encoding = { 
    # ATLANTIC
    "TOR": 1,
    "BOS": 2,
    "NYK": 3, 
    "BRK": 4,
    "PHI": 5,

    # CENTRAL
    "CLE": 6,
    "IND": 7,
    "DET": 8,
    "CHI": 9,
    "MIL": 10,

    # SOUTHEAST
    "MIA": 11,
    "ATL": 12,
    "CHO": 13,
    "WAS": 14,
    "ORL": 15,

    # NORTHWEST
    "OKC": 16,
    "POR": 17,
    "UTA": 18,
    "DEN": 19,
    "MIN": 20,

    # PACIFIC
    "GSW": 21, 
    "LAC": 22,
    "SAC": 23,
    "PHO": 24,
    "LAL": 25,

    # SOUTH WEST
    "SAS": 26,
    "DAL": 27,
    "MEM": 28,
    "HOU": 29,
    "NOP": 30
}

In [5]:
unique_stats = [col.split("_")[0] + "_prev_game" for col in df.columns if "_team0" in col and "restDays" not in col]
stats = [col.split("_")[0] for col in df.columns if "_team0" in col and "restDays" not in col]
teams = team_encoding.keys()

In [6]:
def initialize_prev_game_stats():
    prev_game = {}
    for team in teams:
        team_prev_game = {}
        for stat in unique_stats:
            team_prev_game[stat] = 0
        prev_game[team] = team_prev_game
        prev_game[team]["games_played"] = 0
    
    return prev_game

In [7]:
prev_game = initialize_prev_game_stats()
current_season = None

first_game_indices = set()
    
for index, row in df.iterrows():
    team0 = row['team0']
    team1 = row['team1']
    game_season = row['season']

    if game_season != current_season:
        prev_game = initialize_prev_game_stats()
        current_season = game_season

    games_played_team0 = prev_game[team0]["games_played"]
    games_played_team1 = prev_game[team1]["games_played"]

    if games_played_team0 == 0:
        first_game_indices.add(index)
    
    if games_played_team1 == 0:
        first_game_indices.add(index)

    if games_played_team0 > 0:
        for stat in unique_stats:
            prev_game_team0 = prev_game[team0][stat]
            df.at[index, f"{stat}_team0"] = prev_game_team0
    
    if games_played_team1 > 0:
        for stat in unique_stats:
            prev_avg_team1 = prev_game[team1][stat]
            df.at[index, f"{stat}_team1"] = prev_avg_team1
    
    for stat in stats:
        prev_game[team0][f"{stat}_prev_game"] = row[f"{stat}_team0"]
        prev_game[team1][f"{stat}_prev_game"] = row[f"{stat}_team1"]

    prev_game[team0]['games_played'] += 1
    prev_game[team1]['games_played'] += 1

    if index % 100 == 0:
         print(f"{index} / {len(df)}")

0 / 8468
100 / 8468
200 / 8468
300 / 8468
400 / 8468
500 / 8468
600 / 8468
700 / 8468
800 / 8468
900 / 8468
1000 / 8468
1100 / 8468
1200 / 8468
1300 / 8468
1400 / 8468
1500 / 8468
1600 / 8468
1700 / 8468
1800 / 8468
1900 / 8468
2000 / 8468
2100 / 8468
2200 / 8468
2300 / 8468
2400 / 8468
2500 / 8468
2600 / 8468
2700 / 8468
2800 / 8468
2900 / 8468
3000 / 8468
3100 / 8468
3200 / 8468
3300 / 8468
3400 / 8468
3500 / 8468
3600 / 8468
3700 / 8468
3800 / 8468
3900 / 8468
4000 / 8468
4100 / 8468
4200 / 8468
4300 / 8468
4400 / 8468
4500 / 8468
4600 / 8468
4700 / 8468
4800 / 8468
4900 / 8468
5000 / 8468
5100 / 8468
5200 / 8468
5300 / 8468
5400 / 8468
5500 / 8468
5600 / 8468
5700 / 8468
5800 / 8468
5900 / 8468
6000 / 8468
6100 / 8468
6200 / 8468
6300 / 8468
6400 / 8468
6500 / 8468
6600 / 8468
6700 / 8468
6800 / 8468
6900 / 8468
7000 / 8468
7100 / 8468
7200 / 8468
7300 / 8468
7400 / 8468
7500 / 8468
7600 / 8468
7700 / 8468
7800 / 8468
7900 / 8468
8000 / 8468
8100 / 8468
8200 / 8468
8300 / 8468
8400

In [8]:
first_game_indices

df = df.drop(first_game_indices)

In [9]:
df

,mp_team0,fg_team0,fga_team0,fg%_team0,3p_team0,3pa_team0,3p%_team0,ft_team0,fta_team0,ft%_team0,orb_team0,drb_team0,trb_team0,ast_team0,stl_team0,blk_team0,tov_team0,pf_team0,pts_team0,ts%_team0,efg%_team0,3par_team0,ftr_team0,orb%_team0,drb%_team0,trb%_team0,ast%_team0,stl%_team0,blk%_team0,tov%_team0,ortg_team0,drtg_team0,ft/fga_team0,team0,mp_team1,fg_team1,fga_team1,fg%_team1,3p_team1,3pa_team1,...,ast%_prev_game_team1,stl%_prev_game_team1,blk%_prev_game_team1,tov%_prev_game_team1,ortg_prev_game_team1,drtg_prev_game_team1,ft/fga_prev_game_team1,mp_prev_game_team0,fg_prev_game_team0,fga_prev_game_team0,fg%_prev_game_team0,3p_prev_game_team0,3pa_prev_game_team0,3p%_prev_game_team0,ft_prev_game_team0,fta_prev_game_team0,ft%_prev_game_team0,orb_prev_game_team0,drb_prev_game_team0,trb_prev_game_team0,ast_prev_game_team0,stl_prev_game_team0,blk_prev_game_team0,tov_prev_game_team0,pf_prev_game_team0,pts_prev_game_team0,ts%_prev_game_team0,efg%_prev_game_team0,3par_prev_game_team0,ftr_prev_game_team0,orb%_prev_game_team0,drb%_prev_game_team0,trb%_prev_game_team0,ast%_prev_game_team0,stl%_prev_game_team0,blk%_prev_game_team0,tov%_prev_game_team0,ortg_prev_game_team0,drtg_prev_game_team0,ft/fga_prev_game_team0
16,240.0,44.0,81.0,0.543,11.0,26.0,0.423,17.0,17.0,1.000,9.0,35.0,44.0,25.0,8.0,3.0,17.0,11.0,116.0,0.656,0.611,0.321,0.210,29.0,81.4,59.5,56.8,8.6,6.4,16.1,125.2,104.7,0.210,CLE,240.0,38.0,82.0,0.463,11.0,35.0,...,50.0,6.1,6.3,14.4,110.4,102.2,0.329,240.0,38.0,83.0,0.458,5.0,22.0,0.227,21.0,25.0,0.840,9.0,41.0,50.0,19.0,3.0,4.0,17.0,25.0,102.0,0.543,0.488,0.265,0.301,19.6,82.0,52.1,50.0,3.0,7.1,15.3,102.7,99.7,0.253
17,240.0,48.0,93.0,0.516,12.0,26.0,0.462,24.0,33.0,0.727,5.0,37.0,42.0,22.0,8.0,3.0,18.0,31.0,132.0,0.614,0.581,0.280,0.355,11.1,78.7,45.7,45.8,6.8,4.8,14.3,112.7,110.9,0.258,LAL,240.0,45.0,92.0,0.489,14.0,29.0,...,35.7,9.1,12.1,14.2,77.0,125.6,0.146,240.0,37.0,91.0,0.407,4.0,16.0,0.250,14.0,23.0,0.609,12.0,40.0,52.0,21.0,8.0,7.0,19.0,15.0,92.0,0.455,0.429,0.176,0.253,22.2,70.2,46.8,56.8,7.6,9.5,15.8,86.9,102.0,0.154
18,240.0,47.0,92.0,0.511,18.0,41.0,0.439,16.0,20.0,0.800,10.0,39.0,49.0,29.0,6.0,8.0,18.0,23.0,128.0,0.635,0.609,0.446,0.217,22.2,70.9,49.0,61.7,5.6,12.9,15.2,119.3,111.8,0.174,GSW,240.0,45.0,98.0,0.459,16.0,36.0,...,50.0,4.3,19.0,15.6,97.6,110.4,0.304,240.0,43.0,80.0,0.538,16.0,30.0,0.533,19.0,21.0,0.905,6.0,35.0,41.0,34.0,5.0,9.0,17.0,25.0,121.0,0.678,0.638,0.375,0.263,15.4,77.8,48.8,79.1,4.9,16.1,16.0,118.6,119.6,0.238
19,240.0,45.0,89.0,0.506,18.0,33.0,0.545,13.0,15.0,0.867,9.0,31.0,40.0,30.0,9.0,3.0,22.0,27.0,121.0,0.633,0.607,0.371,0.169,21.4,67.4,45.5,66.7,8.4,4.6,18.7,112.6,117.3,0.146,ORL,240.0,46.0,92.0,0.500,9.0,27.0,...,48.9,6.2,2.9,15.6,115.7,123.6,0.309,240.0,43.0,90.0,0.478,8.0,21.0,0.381,22.0,29.0,0.759,11.0,39.0,50.0,22.0,8.0,9.0,14.0,18.0,116.0,0.564,0.522,0.233,0.322,25.0,78.0,53.2,51.2,7.6,12.7,12.0,110.3,103.6,0.244
20,240.0,35.0,84.0,0.417,10.0,29.0,0.345,22.0,32.0,0.688,14.0,41.0,55.0,16.0,4.0,5.0,19.0,24.0,102.0,0.520,0.476,0.345,0.381,29.8,74.5,53.9,45.7,4.0,8.3,16.2,102.5,92.4,0.262,BOS,240.0,35.0,92.0,0.380,10.0,32.0,...,58.1,5.8,8.0,14.4,110.3,115.1,0.151,240.0,39.0,91.0,0.429,11.0,28.0,0.393,11.0,21.0,0.524,11.0,32.0,43.0,23.0,12.0,2.0,12.0,27.0,100.0,0.499,0.489,0.308,0.231,22.0,84.2,48.9,59.0,12.3,3.6,10.7,102.2,110.4,0.121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8463,240.0,45.0,84.0,0.536,17.0,35.0,0.486,12.0,14.0,0.857,9.0,32.0,41.0,20.0,3.0,6.0,14.0,25.0,119.0,0.660,0.637,0.417,0.167,25.7,74.4,52.6,44.4,3.2,13.3,13.4,127.9,131.1,0.143,MIL,240.0,40.0,85.0,0.471,18.0,40.0,...,69.8,8.6,14.5,8.2,128.2,101.3,0.116,240.0,51.0,94.0,0.543,24.0,41.0,0.585,14.0,14.0,1.000,10.0,25.0,35.0,35.0,11

In [10]:
exceptions = ['team1', 'winner', 'season', 'date', 'team0', 'team0_encoded', 'team1_encoded', 'restDays_team0', 'restDays_team1']

cols_to_keep = [col for col in df.columns if "_prev_game" in col or col in exceptions]

df_filtered = df[cols_to_keep]

In [11]:
df['team1_winner'] = df.apply(lambda row: 1 if row['winner'] == row['team1'] else 0, axis=1)
df_filtered['team1_winner'] = df_filtered.apply(lambda row: 1 if row['winner'] == row['team1'] else 0, axis=1)

/var/folders/rd/xg_1cl914sb4d864cq60_8zr0000gn/T/ipykernel_88459/584802485.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['team1_winner'] = df_filtered.apply(lambda row: 1 if row['winner'] == row['team1'] else 0, axis=1)


In [12]:
df_filtered

,team0,team1,winner,season,date,team0_encoded,team1_encoded,restDays_team0,restDays_team1,mp_prev_game_team1,fg_prev_game_team1,fga_prev_game_team1,fg%_prev_game_team1,3p_prev_game_team1,3pa_prev_game_team1,3p%_prev_game_team1,ft_prev_game_team1,fta_prev_game_team1,ft%_prev_game_team1,orb_prev_game_team1,drb_prev_game_team1,trb_prev_game_team1,ast_prev_game_team1,stl_prev_game_team1,blk_prev_game_team1,tov_prev_game_team1,pf_prev_game_team1,pts_prev_game_team1,ts%_prev_game_team1,efg%_prev_game_team1,3par_prev_game_team1,ftr_prev_game_team1,orb%_prev_game_team1,drb%_prev_game_team1,trb%_prev_game_team1,ast%_prev_game_team1,stl%_prev_game_team1,blk%_prev_game_team1,tov%_prev_game_team1,ortg_prev_game_team1,drtg_prev_game_team1,ft/fga_prev_game_team1,mp_prev_game_team0,fg_prev_game_team0,fga_prev_game_team0,fg%_prev_game_team0,3p_prev_game_team0,3pa_prev_game_team0,3p%_prev_game_team0,ft_prev_game_team0,fta_prev_game_team0,ft%_prev_game_team0,orb_prev_game_team0,drb_prev_game_team0,trb_prev_game_team0,ast_prev_game_team0,stl_prev_game_team0,blk_prev_game_team0,tov_prev_game_team0,pf_prev_game_team0,pts_prev_game_team0,ts%_prev_game_team0,efg%_prev_game_team0,3par_prev_game_team0,ftr_prev_game_team0,orb%_prev_game_team0,drb%_prev_game_team0,trb%_prev_game_team0,ast%_prev_game_team0,stl%_prev_game_team0,blk%_prev_game_team0,tov%_prev_game_team0,ortg_prev_game_team0,drtg_prev_game_team0,ft/fga_prev_game_team0,team1_winner
16,CLE,MIL,CLE,2018,2017-10-20,6,10,2.0,1.0,240.0,38.0,76.0,0.500,7.0,21.0,0.333,25.0,30.0,0.833,6.0,39.0,45.0,19.0,6.0,4.0,15.0,22.0,108.0,0.605,0.546,0.276,0.395,15.8,78.0,51.1,50.0,6.1,6.3,14.4,110.4,102.2,0.329,240.0,38.0,83.0,0.458,5.0,22.0,0.227,21.0,25.0,0.840,9.0,41.0,50.0,19.0,3.0,4.0,17.0,25.0,102.0,0.543,0.488,0.265,0.301,19.6,82.0,52.1,50.0,3.0,7.1,15.3,102.7,99.7,0.253,0
17,LAL,PHO,LAL,2018,2017-10-20,25,24,0.0,1.0,240.0,28.0,89.0,0.315,7.0,27.0,0.259,13.0,18.0,0.722,10.0,23.0,33.0,10.0,9.0,8.0,16.0,22.0,76.0,0.392,0.354,0.303,0.202,19.2,60.5,36.7,35.7,9.1,12.1,14.2,77.0,125.6,0.146,240.0,37.0,91.0,0.407,4.0,16.0,0.250,14.0,23.0,0.609,12.0,40.0,52.0,21.0,8.0,7.0,19.0,15.0,92.0,0.455,0.429,0.176,0.253,22.2,70.2,46.8,56.8,7.6,9.5,15.8,86.9,102.0,0.154,0
18,GSW,NOP,GSW,2018,2017-10-20,21,30,2.0,1.0,240.0,30.0,79.0,0.380,7.0,25.0,0.280,24.0,29.0,0.828,16.0,35.0,51.0,15.0,4.0,12.0,17.0,31.0,91.0,0.496,0.424,0.316,0.367,34.8,70.0,53.1,50.0,4.3,19.0,15.6,97.6,110.4,0.304,240.0,43.0,80.0,0.538,16.0,30.0,0.533,19.0,21.0,0.905,6.0,35.0,41.0,34.0,5.0,9.0,17.0,25.0,121.0,0.678,0.638,0.375,0.263,15.4,77.8,48.8,79.1,4.9,16.1,16.0,118.6,119.6,0.238,0
19,ORL,BRK,BRK,2018,2017-10-20,15,4,1.0,1.0,240.0,45.0,94.0,0.479,12.0,30.0,0.400,29.0,32.0,0.906,11.0,32.0,43.0,22.0,7.0,2.0,20.0,25.0,131.0,0.606,0.543,0.319,0.340,25.0,69.6,47.8,48.9,6.2,2.9,15.6,115.7,123.6,0.309,240.0,43.0,90.0,0.478,8.0,21.0,0.381,22.0,29.0,0.759,11.0,39.0,50.0,22.0,8.0,9.0,14.0,18.0,116.0,0.564,0.522,0.233,0.322,25.0,78.0,53.2,51.2,7.6,12.7,12.0,110.3,103.6,0.244,1
20,BOS,PHI,BOS,2018,2017-10-20,2,5,1.0,1.0,240.0,43.0,93.0,0.462,15.0,35.0,0.429,14.0,19.0,0.737,11.0,37.0,48.0,25.0,6.0,6.0,17.0,28.0,115.0,0.567,0.543,0.376,0.204,23.4,69.8,48.0,58.1,5.8,8.0,14.4,110.3,115.1,0.151,240.0,39.0,91.0,0.429,11.0,28.0,0.393,11.0,21.0,0.524,11.0,32.0,43.0,23.0,12.0,2.0,12.0,27.0,100.0,0.499,0.489,0.308,0.231,22.0,84.2,48.9,59.0,12.3,3.6,10.7,102.2,110.4,0.121,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8463,MIL,BOS,BOS,2024,2024-03-20,10,2,2.0,1.0,240.0,43.0,95.0,0.453,22.0,49.0,0.449,11.0,13.0,0.846,15.0,38.0,53.0,30.0,8.0,8.0,9.0,9.0,119.0,0.591,0.568,0.516,0.137,30.0,82.6,55.2,69.8,8.6,14.5,8.2,128.2,101.3,0.116,240.0,51.0,94.0,0.543,24.0,41.0,0.585,14.0,14.0,1.000,10.0,25.0,35.0,35.0,11.0,2.0

In [13]:
df_filtered.reset_index(drop=True, inplace=True)

In [14]:
df_filtered

,team0,team1,winner,season,date,team0_encoded,team1_encoded,restDays_team0,restDays_team1,mp_prev_game_team1,fg_prev_game_team1,fga_prev_game_team1,fg%_prev_game_team1,3p_prev_game_team1,3pa_prev_game_team1,3p%_prev_game_team1,ft_prev_game_team1,fta_prev_game_team1,ft%_prev_game_team1,orb_prev_game_team1,drb_prev_game_team1,trb_prev_game_team1,ast_prev_game_team1,stl_prev_game_team1,blk_prev_game_team1,tov_prev_game_team1,pf_prev_game_team1,pts_prev_game_team1,ts%_prev_game_team1,efg%_prev_game_team1,3par_prev_game_team1,ftr_prev_game_team1,orb%_prev_game_team1,drb%_prev_game_team1,trb%_prev_game_team1,ast%_prev_game_team1,stl%_prev_game_team1,blk%_prev_game_team1,tov%_prev_game_team1,ortg_prev_game_team1,drtg_prev_game_team1,ft/fga_prev_game_team1,mp_prev_game_team0,fg_prev_game_team0,fga_prev_game_team0,fg%_prev_game_team0,3p_prev_game_team0,3pa_prev_game_team0,3p%_prev_game_team0,ft_prev_game_team0,fta_prev_game_team0,ft%_prev_game_team0,orb_prev_game_team0,drb_prev_game_team0,trb_prev_game_team0,ast_prev_game_team0,stl_prev_game_team0,blk_prev_game_team0,tov_prev_game_team0,pf_prev_game_team0,pts_prev_game_team0,ts%_prev_game_team0,efg%_prev_game_team0,3par_prev_game_team0,ftr_prev_game_team0,orb%_prev_game_team0,drb%_prev_game_team0,trb%_prev_game_team0,ast%_prev_game_team0,stl%_prev_game_team0,blk%_prev_game_team0,tov%_prev_game_team0,ortg_prev_game_team0,drtg_prev_game_team0,ft/fga_prev_game_team0,team1_winner
0,CLE,MIL,CLE,2018,2017-10-20,6,10,2.0,1.0,240.0,38.0,76.0,0.500,7.0,21.0,0.333,25.0,30.0,0.833,6.0,39.0,45.0,19.0,6.0,4.0,15.0,22.0,108.0,0.605,0.546,0.276,0.395,15.8,78.0,51.1,50.0,6.1,6.3,14.4,110.4,102.2,0.329,240.0,38.0,83.0,0.458,5.0,22.0,0.227,21.0,25.0,0.840,9.0,41.0,50.0,19.0,3.0,4.0,17.0,25.0,102.0,0.543,0.488,0.265,0.301,19.6,82.0,52.1,50.0,3.0,7.1,15.3,102.7,99.7,0.253,0
1,LAL,PHO,LAL,2018,2017-10-20,25,24,0.0,1.0,240.0,28.0,89.0,0.315,7.0,27.0,0.259,13.0,18.0,0.722,10.0,23.0,33.0,10.0,9.0,8.0,16.0,22.0,76.0,0.392,0.354,0.303,0.202,19.2,60.5,36.7,35.7,9.1,12.1,14.2,77.0,125.6,0.146,240.0,37.0,91.0,0.407,4.0,16.0,0.250,14.0,23.0,0.609,12.0,40.0,52.0,21.0,8.0,7.0,19.0,15.0,92.0,0.455,0.429,0.176,0.253,22.2,70.2,46.8,56.8,7.6,9.5,15.8,86.9,102.0,0.154,0
2,GSW,NOP,GSW,2018,2017-10-20,21,30,2.0,1.0,240.0,30.0,79.0,0.380,7.0,25.0,0.280,24.0,29.0,0.828,16.0,35.0,51.0,15.0,4.0,12.0,17.0,31.0,91.0,0.496,0.424,0.316,0.367,34.8,70.0,53.1,50.0,4.3,19.0,15.6,97.6,110.4,0.304,240.0,43.0,80.0,0.538,16.0,30.0,0.533,19.0,21.0,0.905,6.0,35.0,41.0,34.0,5.0,9.0,17.0,25.0,121.0,0.678,0.638,0.375,0.263,15.4,77.8,48.8,79.1,4.9,16.1,16.0,118.6,119.6,0.238,0
3,ORL,BRK,BRK,2018,2017-10-20,15,4,1.0,1.0,240.0,45.0,94.0,0.479,12.0,30.0,0.400,29.0,32.0,0.906,11.0,32.0,43.0,22.0,7.0,2.0,20.0,25.0,131.0,0.606,0.543,0.319,0.340,25.0,69.6,47.8,48.9,6.2,2.9,15.6,115.7,123.6,0.309,240.0,43.0,90.0,0.478,8.0,21.0,0.381,22.0,29.0,0.759,11.0,39.0,50.0,22.0,8.0,9.0,14.0,18.0,116.0,0.564,0.522,0.233,0.322,25.0,78.0,53.2,51.2,7.6,12.7,12.0,110.3,103.6,0.244,1
4,BOS,PHI,BOS,2018,2017-10-20,2,5,1.0,1.0,240.0,43.0,93.0,0.462,15.0,35.0,0.429,14.0,19.0,0.737,11.0,37.0,48.0,25.0,6.0,6.0,17.0,28.0,115.0,0.567,0.543,0.376,0.204,23.4,69.8,48.0,58.1,5.8,8.0,14.4,110.3,115.1,0.151,240.0,39.0,91.0,0.429,11.0,28.0,0.393,11.0,21.0,0.524,11.0,32.0,43.0,23.0,12.0,2.0,12.0,27.0,100.0,0.499,0.489,0.308,0.231,22.0,84.2,48.9,59.0,12.3,3.6,10.7,102.2,110.4,0.121,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8350,MIL,BOS,BOS,2024,2024-03-20,10,2,2.0,1.0,240.0,43.0,95.0,0.453,22.0,49.0,0.449,11.0,13.0,0.846,15.0,38.0,53.0,30.0,8.0,8.0,9.0,9.0,119.0,0.591,0.568,0.516,0.137,30.0,82.6,55.2,69.8,8.6,14.5,8.2,128.2,101.3,0.116,240.0,51.0,94.0,0.543,24.0,41.0,0.585,14.0,14.0,1.000,10.0,25.0,35.0,35.0,11.0,2.0,10.0

In [15]:
save_df(df_filtered, "prev_game_df.csv")